In [ ]:
import pandas as pd
import psycopg2
import uuid
import bcrypt
import string
import random

#### Clean data columns

In [4]:
data = pd.read_csv('search.csv')

In [5]:
# Build array of bachhas
all_bachhas = []

for index, row in data.iterrows():
    bachhas_list = []
    for i in range(12):
        col_name = f'c[{i}]'
        if col_name in row and pd.notna(row[col_name]):
            try:
                bachhas_list.append(int(row[col_name]))
            except:
                bachhas_list.append(row[col_name])
    all_bachhas.append(bachhas_list)
data['bachhas'] = all_bachhas


In [6]:
# clean up extra cols
data.drop(columns=data.columns[13:-2], inplace=True)
data.drop(columns=["_id__baas_transaction"], inplace=True)
data.columns

Index(['_id', 'a', 'b', 'd', 'g', 'h', 'i', 'n', 'p', 'r', 'u', 's', 'c',
       'bachhas'],
      dtype='object')

In [12]:
data.to_csv('clean_data.csv', index=False)

In [ ]:
data_read = pd.read_csv('clean_data.csv', index_col="_id")
data_read.head()

### Save data into db

In [15]:
def generate_random_password(length = 10):
    characters = string.ascii_letters + string.digits + string.punctuation
    random_string = ''.join(random.choices(characters, k=length))
    return random_string

In [ ]:
# Connect to PSQL
from datetime import datetime


conn = psycopg2.connect( # edit these to match the Locations table
    dbname="compass",
    user="this_is_mjk",
    password="",
    host="",
    port=5432
)
cursor = conn.cursor()


passwords = {}
empty_email = []
error_students = []

for index, student in data.iterrows():
    if not index % 100:
        print(index)
    user_id = str(uuid.uuid4())
    password = generate_random_password()
    passwords[student['i']] = password
    if pd.isna(student['u']):
        empty_email.append(student)
        continue
    try:
        cursor.execute("""
            INSERT INTO users (
                user_id,
                email,
                password,
                is_verified,
                role
            )
            VALUES (%s, %s, %s, %s, %s)
        """, (
            datetime.now(),                 # created_at
            datetime.now(),                 # updated_at
            None,                           # deleted_at
            user_id,                        # user_id (uuid.UUID)
            student['u'] + "@iitk.ac.in",   # email
            bcrypt.hashpw(password.encode(),  bcrypt.gensalt()),
            True,
            50
        ))
        cursor.execute("""
            INSERT INTO profiles (
                user_id,
                name,
                email,
                roll_no,
                dept,
                course,
                gender,
                hall,
                room_number,
                home_town,
                visibility,
                bapu,
                bachhas
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            datetime.now(),                 # created_at
            datetime.now(),                 # updated_at
            None,                           # deleted_at
            user_id,                        # user_id (uuid.UUID)
            student["n"],
            student['u'] + "@iitk.ac.in",   # email
            student["i"],
            student["d"],
            student["c"],
            student["g"],
            student["h"],
            student["r"],
            student["s"],
            student["a"],
            random.choice([True, False]),
            student["bachhas"],
        ))
        conn.commit()
    except Exception as e:
        conn.rollback()
        error_students.append({"s": student, "e": e})
        print(f"Error on student: {student['n']} ({student['u']}). Reason: {e}")
        continue

In [18]:
conn.close()

### Save users with issues

In [ ]:
print(len(error_students), len(empty_email))

In [ ]:
pd.DataFrame(empty_email).to_csv("empty_email.csv", index=False)
len(empty_email)

In [30]:
list = []
for obj in error_students:
    obj['s']['error'] = obj['e']
    list.append(obj['s'])
pd.DataFrame(list).to_csv("repeat.csv", index=False)